# Tables

## Create Table
- https://sqlite.org/lang_createtable.html
- CREATE TABLE statement is used to define and create a new table within a database
- a table is a structured collection of data organized into rows and columns, where each column has a specific data type and each row represents a record
- keywords within `[ ]` are optional
- syntax

```sql
CREATE TABLE [IF NOT EXISTS] [schema_name].table_name (
    column_1 data_type PRIMARY KEY,
    column_2 data_type NOT NULL,
    column_3 data_type DEFAULT 0,
    ...
    table_constraints
) [WITHOUT ROWID];
```
- Sqlite automatically adds ROWID AUTOINCREMENT column for every table created
- you can tell it to not do that by using [WITHOUT ROWID] optional clause
- e.g.,

```sql
CREATE TABLE employees (
    EmployeeID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    FirstName VARCHAR(50) NOT NULL,
    LastName VARCHAR(50) NOT NULL,
    DateOfBirth DATETIME
);
```

- use DBeaver to run the script on any chinook sqlite database

## Table Definition

- tables are the primary data storage units
- each table is characterized by its name, columns, data types, and constraints
- columns (also called attributes or fields) are data/values stored in each record
- tables need to be created before data can be stored into it

## Column Definition

- each table definition specifies the columns (also known as fields or attributes)
- column definitions include the column name, data type, and any constraints
- most Relational DB Engines use static/rigid data types for columns
- SQLite DB uses flexible dynamic typing and it's a feature not bug
- SQLite provides STRICT Table for static/rigid typing enforcement

### SQLite Storage Classes & Datatypes

- https://sqlite.org/datatype3.html
- each value stored in an SQLite database (or manipulated by the database engine) has one of the following storage classes:

| Storage | Datatype |
| ----- | ----------|
| NULL | NULL value |
| INTEGER | signed integer in 0...8 bytes depending on the magnitude |
| REAL | floating point value 8-byte |
| TEXT | text string (utf-8 and other encodings)
| BLOB | blob of data, stored exactly as it as input |

### Date and Time Datatype

- SQLite does not have a storage class set aside for storing dates and/or times
- usually dates and times are stored as TEXT, REAL, or INTEGER
- use SQLite provided functions or language features to convert data to date and time

## Table constraints
- PRIMARY KEY, FOREIGN KEY, UNIQUE, CHECK

## Primary Keys

- a primary key is a unique identifier for a record within a relational database table
- is used to uniquely identify each row or record in a table
- the primary key ensures that there are no duplicate values within the column(s) designated as the primary key
- primary keys help to maintain data integrity and provide a reliable and efficient way to access and manage data
- primary key is used to establish relationship among the tables
- CustomerID column in the table above is the primary key
- can use separate line with PRIMARY KEY constraint to identify PK column(s), especially if there are more than 1 primary key column

- e.g.: 

```sql
CREATE TABLE employee (
    EmployeeID INTEGER NOT NULL,
    FirstName VARCHAR(50) NOT NULL,
    LastName VARCHAR(50) NOT NULL,
    DateOfBirth DATETIME,
    PRIMARY KEY (EmployeeID)
);
```
- can also use a list of multiple columns as PRIMARY KEY(column_1, column_2,...)

### Key characteristics of Primary Key

#### Uniquness
 - a primary key must contain unique values for each record in the table
- No two records can have the same primary key value

#### Non-Null
- values in a primary key column cannot be null (empty) because null values are not unique

#### Single or Composite
- can consist of a single column or multiple columns, depending on the requirements of the database design

#### Data Integrity
- by enforcing uniqueness and non-null constraints, the primary key ensures that each record is uniquely identifiable, preventing data duplication and inconsistency

#### Access and Referencing
- primary keys are used to uniquely identify records within the table
- they also serve as a basis for establishing relationships between tables using foreign keys.

#### Automatically Generated Primary Keys
- in some cases, primary key values are automatically generated by the DBMS when new records are added
- this is often seen with auto-incrementing integer values


## FOREIGN KEY

- FK is a field or a set of fields that refers to the primary key of another table
- it establishes a relationship between two tables, enabling you to maintain referential integrity and ensure data consistency across related tables
- in essence, a foreign key creates a link between records in two tables, enforcing rules about the relationships between those records
- the table containing the foreign key is referred to as the "child" table, and the table to which the foreign key refers is referred to as the "parent" table
- the foreign key column(s) in the child table hold values that match the primary key values in the parent table
- when inserting record into child table, there must be an existing record in the parent table

### Benefits of using foreign keys include

#### Referential Integrity
- Foreign keys help maintain the integrity of the relationships between tables, preventing orphaned or inconsistent data

#### Data Consistency
- they ensure that only valid values are inserted into the foreign key column, reducing the risk of data anomalies

#### Relationship Definition
- Foreign keys explicitly define the relationships between tables, making it easier to understand the database's structure
- help reduce data redundancy making tables smaller storing unique information on some entity
- run the following statements one at a time as Sqlite doesn't allow multiple statements at once

```sql
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name VARCHAR(100),
    email VARCHAR(255)
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    order_date DATETIME,
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    /*FOREIGN KEY ...*/
);
```

### FOREIGN KEY Constraints

- you can add constraints to FOREIGN KEY
- e.g.,
```sql
FOREIGN KEY (foreign_key_columns)
   REFERENCES parent_table(parent_key_columns)
      ON UPDATE action 
      ON DELETE action
```

- rules that are defined on a column or a set of columns to enforce referential integrity between related tables when related records in parent table are updated or deleted 

Here are some common types of foreign key constraints/actions:

1. **CASCADE**: 
- when a record in the parent table is deleted or updated, the corresponding records in the child table are also deleted or updated automatically
- this helps maintain data consistency across related tables

```sql
FOREIGN KEY (foreign_key_columns)
   REFERENCES parent_table(parent_key_columns)
      ON DELETE CASCADE
```

2. **SET NULL**:
- when a record in the parent table is deleted or updated, the foreign key values in the child table are set to `NULL`
- this is useful when you want to allow records to exist in the child table even if the related record in the parent table is deleted

```sql
FOREIGN KEY (foreign_key_columns)
   REFERENCES parent_table(parent_key_columns)
      ON DELETE SET NULL
```

3. **SET DEFAULT**: 
- similar to `SET NULL`, but the foreign key values in the child table are set to their default values defined when the table was created

```sql
FOREIGN KEY (foreign_key_columns)
   REFERENCES parent_table(parent_key_columns)
      ON DELETE SET DEFAULT
```

4. **NO ACTION**:
- prevents any actions that would violate referential integrity, such as deleting a record from the parent table if related records exist in the child table
- this constraint essentially stops actions that would create inconsistencies

```sql
FOREIGN KEY (foreign_key_columns)
   REFERENCES parent_table(parent_key_columns)
      ON DELETE CASCADE
      ON UPDATE NO ACTION
```

5. **RESTRICT**: 
- similar to `NO ACTION`, it prevents actions that would violate referential integrity
- tt's a more restrictive version of the constraint and is often used interchangeably with `NO ACTION`

6. **CHECK**:
- CHECK constraints allow you to define additional data integrity checks beyond UNIQUE or NOT NULL to suit your specific application
- allows you to define a boolean expressions to test values whenever they are inserted into or updated within a column
- if the values do not meet the criteria defined by the expression, SQLite will issue a constraint violation and abort the statement
- SQLite allows you to define a CHECK constraint at the column level or the table level
- syntax of CHECK constraint at the column level:

```sql
CREATE TABLE table_name(
    ...,
    column_name data_type CHECK(expression),
    ...
);
```
- e.g. test the following table in a Sqlite database:
```sql
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    middle_initial TEXT NOT NULL 
            CHECK (length(middle_initial) == 1)
);
```

- CHECK constraint at the table level example:

```sql
CREATE TABLE products (
    product_id   INTEGER         PRIMARY KEY,
    product_name TEXT            NOT NULL,
    list_price   DECIMAL (10, 2) NOT NULL,
    discount     DECIMAL (10, 2) NOT NULL DEFAULT 0,
    CHECK (list_price >= discount AND 
        discount >= 0 AND 
        list_price >= 0) 
);
```

### Constraints Example

Here's an example of how you might apply these constraints in SQL:

```sql
CREATE TABLE authors (
    author_id INTEGER PRIMARY KEY
        CHECK(author_id >= 1000),
    author_name TEXT
);

CREATE TABLE books (
    book_id INTEGER PRIMARY KEY,
    book_title TEXT,
    author_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
        ON DELETE CASCADE
);

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    order_date DATETIME,
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        ON DELETE SET NULL
);
```

In this example:
- the `books` table has a foreign key constraint with `ON DELETE CASCADE`, meaning if an author is deleted from the `authors` table, all related books will be deleted as well
- the `orders` table has a foreign key constraint with `ON DELETE SET NULL`, so if a customer is deleted, the customer ID in the orders table will be set to `NULL`.

### Lab

- create the above tables in a Sqlite Database 
- insert some records into all the tables
- delete some records from the parent tables
- observe the records in the child tables


## DROP TABLE

- `DROP TABLE` statement is used to delete a table and remove it from the database schema
- removes both the table structure and all the data stored within it
- https://sqlite.org/lang_droptable.html
- syntax:

```sql
DROP TABLE [IF EXISTS] [schema_name.]table_name;
```

- `IF EXISTS`: This optional clause is used to check if the table exists before attempting to drop it. If the table does not exist, no error will be raised.
    - it's good practice to use the `IF EXISTS` clause to avoid errors if you attempt to drop a table that doesn't exist

- e.g.,

```sql
DROP TABLE IF EXISTS employees;
```

## ALTER TABLE

- https://sqlite.org/lang_altertable.html
- `ALTER TABLE` statement is used to modify an existing table's structure
- it allows you to add, modify, or drop columns in a table, as well as make other changes to the table's definition
- SQLite's support for altering tables is more limited compared to some other database systems
- always back up your data and carefully plan any table alterations to ensure data integrity and minimize disruptions
- basic syntax:

```sql
ALTER TABLE table_name
action;
```

### Common alter table statements

1. **Adding a Column**:
```sql
ALTER TABLE table_name
ADD COLUMN new_column_name data_type;
```
- e.g.,

```sql
ALTER TABLE customers
ADD COLUMN email TEXT;
```

2. **Renaming a Table**:
- syntax

```sql
ALTER TABLE existing_table
RENAME TO new_table;
```
- e.g.,

```
ALTER TABLE customers
RENAME TO clients;
```

3. **Renaming a Columns**:
- syntax

```sql
ALTER TABLE table_name
RENAME COLUMN current_name TO new_name;
```
- e.g.,

```sql
ALTER TABLE products
RENAME discount TO sale;
```

3. **Dropping a Column**:
- can't drop column that is part of some constraints: PRIMARY KEY, CHECK, FOREIGN KEY, etc.

- syntax:
```
ALTER TABLE table_name
DROP COLUMNN column_name;
```
- e.g.,

```sql
ALTER TABLE products
DROP COLUMN product_name;
```

4. **Modifying a Column**:
- SQLite does not support directly modifying column definitions (such as changing data types or constraints) using the `ALTER TABLE` statement
- You usually have to create a new table with the desired modifications and then copy the data.
